# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


## Task 1 :
### Getting the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4




In [8]:


query = "CREATE TABLE IF NOT EXISTS  artists_songs "
query = query + "(sessionid text,iteminsession text,artist text,song_title text,song_length text,PRIMARY KEY (sessionid,iteminsession))"


"""The query creates a table that contains artist,song title and song length.
    The Composite PRIMARY KEY is sessionid and iteminsession because the task is to filter by these fields.
    """

try:
    session.execute(query)
except Exception as e:
    print(e)


                    

In [9]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
       
       
        query = "insert into udacity.artists_songs (sessionid,iteminsession,artist,song_title,song_length)"
        query = query + "VALUES (%s, %s, %s, %s,%s)"
        
        """The query insert data from csv files to the cassandra table
        """
        
  
        session.execute(query,(line[8],line[3],line[0],line[9],line[5]))

In [10]:

query = "select sessionid,iteminsession,artist,song_title,song_length from artists_songs "
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

"""The query extracts all data from the table to check that the process is correct"""
 
for row in rows:
    print (row.sessionid,row.iteminsession,row.artist,row.song_title,row.song_length)


169 1 Fall Out Boy Nobody Puts Baby In The Corner 200.72444
169 2 M.I.A. Mango Pickle Down River (With The Wilcannia Mob) 233.7171
9 0 Black Eyed Peas Pump It 214.93506
139 1 Des'ree You Gotta Be 246.30812
139 3 Mr Oizo Flat 55 144.03873
139 4 Tamba Trio Quem Quiser Encontrar O Amor 177.18812
139 5 The Mars Volta Eriatarka 380.42077
139 6 Infected Mushroom Becoming Insane 440.2673
139 7 Blue October / Imogen Heap Congratulations 241.3971
139 8 Girl Talk Once again 160.15628
100 0 Survivor Eye Of The Tiger 245.36771


## Task 2 -

### Getting the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    



In [11]:

query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "( userid text,sessionid text,iteminsession text,artist text,song_title text, user_name text,PRIMARY KEY ((userid, sessionid), iteminsession))"


"""The query creates a table that contains name of artist,song and user
    The Composite PRIMARY KEY is userid and sessionid because the task is to filter by these fields.
    In addition, the query creates sorting by itemInSession using CLUSTERING COLUMN called itemInSession.
    """

try:
    session.execute(query)
except Exception as e:
    print(e)



with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
       
       
        query = "insert into udacity.song_playlist_session (userid,sessionid,iteminsession,artist,song_title,user_name)"
        query = query + "VALUES (%s, %s, %s, %s,%s,%s)"
        
        session.execute(query,(line[10],line[8],line[3],line[0],line[9],line[1]+" "+line[4]))
        
        
"""The query insert data from csv files to the cassandra table
            """



query = "select userid,artist,song_title,user_name from song_playlist_session where userid = '10' and sessionid = '182' "

"""
The query extracts all filtered data by the conditions userid = '10' and sessionid = '182'"""

try:
    rows=session.execute(query)
except Exception as e:
    print(e)


 
for row in rows:
    print (row.userid,row.artist,row.song_title,row.user_name)

                    

## Task 3 -
### Getting the  name (first and last) in  music app history who listened to the song 'All Hands Against His Own'


In [12]:



query = "CREATE TABLE IF NOT EXISTS udacity.user_song_usage_history"
query = query + "(song_title text,user_name text,PRIMARY KEY (song_title,user_name))"

"""The query creates a table that contains name of song_title and user_name
    The Composite PRIMARY KEY is song_title and user_name because the task is to filter by these fields.
    """

try:
    session.execute(query)
except Exception as e:
    print(e)




with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
       
       
        query = "insert into udacity.user_song_usage_history (song_title,user_name)"
        query = query + "VALUES (%s, %s);"
        session.execute(query,(line[9],line[1]+"' '"+line[4]))
            
            
"""The query insert data from csv files to the cassandra table
"""
        

query = "select song_title,user_name from user_song_usage_history where song_title = 'All Hands Against His Own' "


"""
The query extracts all filtered data by the condition song_title = 'All Hands Against His Own'"""
try:
    rows=session.execute(query)
except Exception as e:
    print(e)


for row in rows:
    print (row.song_title,row.user_name)

                    
                    

SyntaxError: EOL while scanning string literal (<ipython-input-12-89f1b8a29a52>, line 38)

### Drop the tables before closing out the sessions

In [ ]:
query = "drop table udacity.artists_songs"


try:
    session.execute(query)
except Exception as e:
    print(e)
    
    
    
query2 = "drop table udacity.song_playlist_session"


try:
    session.execute(query2)
except Exception as e:
    print(e)
    
    
    
    
query3 = "drop table udacity.user_song_usage_history"


try:
    session.execute(query3)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()